In [1]:
import numpy as np
from numpy import *
import jieba
import jieba.posseg as pseg
import pandas as pd
from keras.utils import to_categorical
df = pd.read_csv('./sohu.csv')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
'''X=train_df.loc[:,'text']
y=train_df.loc[:,'label']
t=0
X_1=[]
for i in X:
    X_1.append(jieba.cut(i))
    t+=1
X_1=filter(lambda x:x !='x', X_1)'''
df['rtext'] = df.text.apply(lambda x: " ".join(jieba.cut(x)))
df = df[['rtext', 'label']]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\JINYIN~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.295 seconds.
Prefix dict has been built succesfully.


In [3]:
df.head()

,rtext,label
0,高清 ： 彭帅 谢淑薇 遗憾 遭 逆转 出局 击掌 互相 鼓励 责编 : 马克 杰 ...,pic
1,河南 周口 路边 秸秆 燃烧 2 岁 幼儿 被 熏 晕 ( 图 ) 10 月 3...,news
2,消息 称 138 名 中国 劳工 在 菲律宾 被 抓 使馆 正 核实 【 使馆 正 核实...,news
3,越南 重要 军事 领导人 武元甲 大将 去世 享年 102 岁 据 美国 媒体 10 月...,news
4,西沙 搜救 已 发现 6 具 遇难 渔民 遗体 尚有 52 人 失踪 【 已 发现 6 ...,news


In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [5]:
maxlen = 100
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df.rtext)
sequences = tokenizer.texts_to_sequences(df.rtext)
data = pad_sequences(sequences, maxlen=maxlen)

word_index = tokenizer.word_index


In [6]:
t=0
for i in df.label:
    if i == 'pic':
        df.label[t] = 0
    if i == 'news':
        df.label[t] = 1
    if i == 'sports':
        df.label[t] = 2
    if i == 'business':
        df.label[t] = 3
    if i == 'mil':
        df.label[t] = 4
    if i == 'caipiao':
        df.label[t] = 5
    if i == 'yule':
        df.label[t] = 6
    if i == 'cul':
        df.label[t] = 7
    t+=1
labels = to_categorical(np.asarray(df.label))

In [7]:
MAX_SEQUENCE_LENGTH = 100 # 每条新闻最大长度
EMBEDDING_DIM = 200 # 词向量空间维度
VALIDATION_SPLIT = 0.16 # 验证集比例
TEST_SPLIT = 0.2 # 测试集比例

In [8]:
p1 = int(len(data)*(1-VALIDATION_SPLIT-TEST_SPLIT))
p2 = int(len(data)*(1-TEST_SPLIT))
x_train = data[:p1]
y_train = labels[:p1]
x_val = data[p1:p2]
y_val = labels[p1:p2]
x_test = data[p2:]
y_test = labels[p2:]


In [9]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import LSTM, Embedding
from keras.models import Sequential

In [10]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, 
          input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 200)          22535600  
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 1608      
Total params: 22,858,008
Trainable params: 22,858,008
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
print (model.metrics_names)
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128)
model.save('lstm.h5')

print (model.evaluate(x_test, y_test))

['loss', 'acc']
Instructions for updating:
Use tf.cast instead.
Train on 3789 samples, validate on 947 samples
Epoch 1/2
3789/3789 [==============================] - 64s 17ms/step - loss: 1.4323 - acc: 0.5329 - val_loss: 1.2795 - val_acc: 0.5692
Epoch 2/2
1185/1185 [==============================] - 7s 6ms/step
[0.9099140395595051, 0.6388185654008439]
